In [ ]:
!pip install transformers

In [ ]:
import numpy as np 
import pandas as pd 

import matplotlib.pyplot as plt

import torch
from tqdm.notebook import tqdm

from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split

from transformers import BertTokenizer
from transformers import BertForSequenceClassification
from transformers import AdamW, get_linear_schedule_with_warmup

from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

In [ ]:
df = pd.read_csv("train.tsv", names=['Relation','Sentence','NER1','NER2'], sep="\t")
df['id'] = range(len(df))
df.set_index('id', inplace=True)
df.head()

In [ ]:
df.Relation.value_counts()

In [ ]:
# Convert relation labels to integer values
labels = df.Relation.unique()

label_dict = {label:index for index, label in enumerate(labels)}

In [ ]:
df['Relation'] = df['Relation'].map(label_dict)
df.head()

In [ ]:
# df_val = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/DL4NLP/data/valid.tsv", names=['Relation','Sentence','NER1','NER2'], sep="\t")
# df_val['Relation'] = df_val['Relation'].map(label_dict)

X_train, X_val, y_train, y_val = train_test_split(df.index.values, df.Relation.values, test_size=0.01, random_state=42)

In [ ]:
# here they also add a label to indicate whether it is a training, validation or test set item.
# if you do cross-validation, you do not need such labeling.
df['data_type'] = ['not_set']*df.shape[0]

df.loc[X_train, 'data_type'] = 'train'
df.loc[X_val, 'data_type'] = 'val'

df.groupby(['Relation', 'data_type']).count()

In [ ]:
df.describe()

In [ ]:
def add_special_tokens(self):
        special_tokens_dict = {'additional_special_tokens': ['<e1>', '</e1>', '<e2>', '</e2>']}
        self.add_special_tokens(special_tokens_dict)
        self.model.resize_token_embeddings(len(self.tokenizer))

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-uncased')

In [ ]:
special_tokens_dict = {'additional_special_tokens': ['<e1>', '</e1>', '<e2>', '</e2>']}
tokenizer.add_special_tokens(special_tokens_dict)

In [ ]:
df.Sentence = df.Sentence.apply(lambda x: "".join(['<s>', x, '</s>']))

In [ ]:
encoded_data_train = tokenizer.batch_encode_plus(
    df.Sentence.values,
    add_special_tokens=False,
    return_attention_mask=True,
    pad_to_max_length=True,
    max_length=256,
    return_tensors='pt'
)

encoded_data_val = tokenizer.batch_encode_plus(
    df.Sentence.values,
    add_special_tokens=False,
    return_attention_mask=True,
    pad_to_max_length=True,
    max_length=256,
    return_tensors='pt'
)

input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(df.Relation.values)

input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(df.Relation.values)

In [ ]:
encoded_data_val['input_ids'].shape

In [ ]:
dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)

In [ ]:
model = BertForSequenceClassification.from_pretrained('bert-base-multilingual-uncased', num_labels=len(label_dict), output_attentions=False, output_hidden_states=False)

In [ ]:
batch_size = 8

dataloader_train = DataLoader(dataset_train, sampler=RandomSampler(dataset_train), batch_size=batch_size)
dataloader_val = DataLoader(dataset_val, sampler=RandomSampler(dataset_val), batch_size=8)

In [ ]:
optimizer = AdamW(model.parameters(), lr=1e-5, eps=1e-8)

In [ ]:
epochs = 5

scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=len(dataloader_train)*epochs)

In [ ]:
def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average='weighted')

In [ ]:
def accuracy_per_class(preds, labels):
    label_dict_inverse = {v:k for k,v in label_dict.items()}

    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        print(f'Class: {label_dict_inverse[label]}')
        print(f'Accuracy: {len(y_preds[y_preds==label])}/{len(y_true)}\n')

In [ ]:
import random

seed_val = 123
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
print(device)

In [ ]:
def evaluate(dataloader_val):

    model.eval()
    
    loss_val_total = 0
    predictions = []
    true_vals = []
    
    for batch in tqdm(dataloader_val):
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2]
                 }

        with torch.no_grad():        
            outputs = model(**inputs)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals

In [ ]:
torch.cuda.empty_cache()

In [ ]:
 model.resize_token_embeddings(len(tokenizer))

In [ ]:
for epoch in tqdm(range(1, epochs+1)):
    model.train()
    loss_train_total = 0
    
    progress_bar = tqdm(dataloader_train, 
                        desc='Epoch {:1d}'.format(epoch), 
                        leave=False, 
                        disable=False)
    
    for batch in progress_bar:
        model.zero_grad()
        batch = tuple(b.to(device) for b in batch)
        inputs = {
            'input_ids':        batch[0],
            'attention_mask':   batch[1],
            'labels':           batch[2],
        }
        
        outputs = model(**inputs)
        loss = outputs[0]
        loss_train_total +=loss.item()
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        
        optimizer.step()
        scheduler.step()
        
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})     
    
    tqdm.write(f'\nEpoch {epoch}')
    
    loss_train_avg = loss_train_total/len(dataloader_train)
    tqdm.write(f'Training loss: {loss_train_avg}')
    
    val_loss, predictions, true_vals = evaluate(dataloader_val)
    val_f1 = f1_score_func(predictions, true_vals)
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'F1 Score (weighted): {val_f1}')

In [ ]:
accuracy_per_class(predictions, true_vals)

-----------------------------------------------------------------------------
-----------------------------------------------------------------------------
-----------------------------------------------------------------------------

In [ ]:
dfv = pd.read_csv("valid.tsv", sep="\t")
dfv['id'] = range(len(dfv))
dfv.set_index('Id', inplace=True)
dfv.head()

In [ ]:
dfv.Sentence = dfv.Sentence.apply(lambda x: "".join(['<s>', x, '</s>']))

In [ ]:
encoded_data_val = tokenizer.batch_encode_plus(
    dfv.Sentence.values,
    add_special_tokens=False,
    return_attention_mask=True,
    pad_to_max_length=True,
    max_length=256,
    return_tensors='pt'
)

input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']


dataset_val = TensorDataset(input_ids_val, attention_masks_val)
dataloader_val = DataLoader(dataset_val, sampler=None, batch_size=8)

In [ ]:
def evaluate_val(dataloader_val):

    model.eval()
    predictions = []
  
    for batch in tqdm(dataloader_val):
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1]                  
                 }

        with torch.no_grad():        
            outputs = model(**inputs)

        
        logits = outputs[0]
 

        logits = logits.detach().cpu().numpy()
        predictions.append(logits)
    
    
    predictions = np.concatenate(predictions, axis=0)
            
    return predictions

In [ ]:
preds = evaluate_val(dataloader_val)

In [ ]:
preds2 = np.argmax(preds,axis=1)
preds2

In [ ]:
label_dict_rev = dict((v,k) for k,v in label_dict.items())

In [ ]:
dfv['preds'] = preds2

In [ ]:
dfv['preds'] = dfv['preds'].map(label_dict_rev)

In [ ]:
dfv

In [ ]:
dfv.to_csv('team_uhh_predictions.csv')